# Scrapping de noticias
<p>En este codigo se realiza el scrapping inicial de los portales seleccionados. En cada uno de los n medios se extráen las noticias de las últimas 10 páginas, existiendo k noticias en cada una, generalmente fija. </p>


In [1]:
# Librerías y setup incial
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime

# Analizar nombres de hojas
for hoja in pd.ExcelFile('../data/enlaces_para_scrappear.xlsx').sheet_names:
    print(hoja)

# 1. Leer el listado de secciones desde un Excel
df_secciones = pd.read_excel('../data/enlaces_para_scrappear.xlsx', sheet_name = 'Hoja1')  # Cambiá el nombre si hace falta
df_secciones.head()



elheraldo_politica_completo
Hoja1


,departamento,portal,seccion,link
0,parana,el diario,noticias,https://www.eldiario.com.ar/category/actualida...
1,parana,el diario,economia,https://www.eldiario.com.ar/category/economia/
2,parana,analisis digital,provinciales,https://www.analisisdigital.com.ar/provinciales
3,parana,analisis digital,economia,https://www.analisisdigital.com.ar/economia
4,concorida,el heraldo,politica,https://www.elheraldo.com.ar/noticias/politica


## El diario

In [2]:
def scrapear_listado_eldiario(url_seccion, seccion, n_pagina):
    from bs4 import BeautifulSoup
    import requests

    url = url_seccion if n_pagina == 1 else f"{url_seccion}page/{n_pagina}/"
    print(f"Scrapeando {seccion} página {n_pagina} -> {url}")
    try:
        res = requests.get(url, timeout=10)
        soup = BeautifulSoup(res.text, 'html.parser')
        items = soup.find_all('div', class_='jl_li_in')
        datos = []
        for item in items:
            # Enlace a la nota
            a = item.find('a', href=True)
            enlace = a['href'] if a else ''
            if enlace and not enlace.startswith('http'):
                enlace = 'https://www.eldiario.com.ar' + enlace
            # Título
            titulo_tag = item.find(class_='jl_fe_title')
            titulo = titulo_tag.get_text(strip=True) if titulo_tag else ''
            # Fecha de publicación
            post_date = item.find(class_='post-date')
            fecha = post_date.get_text(strip=True) if post_date else ''
            datos.append({
                'enlace': enlace,
                'seccion': seccion,
                'pagina': n_pagina,
                'fecha_publicacion': fecha,
                'titulo': titulo
            })
        return datos
    except Exception as e:
        print(f"Error: {e}")
        return []


def scrapear_detalle_eldiario(url_nota, titulo_nota=None):
    """
    Scrapea una nota de El Diario de Paraná y devuelve:
    - descripcion (todo el cuerpo)
    - vistas
    - categorias útiles (tarjetitas/chips)
    """
    import requests
    from bs4 import BeautifulSoup

    IGNORAR = ['Home', 'Editor', 'Mins read', 'Views', 'Editor12 mayo, 20251 Mins read867 Views']

    def es_categoria_util(txt, titulo):
        if not txt or txt in IGNORAR:
            return False
        if titulo and txt == titulo:
            return False
        if txt.lower().startswith("editor") or "mins read" in txt.lower() or "views" in txt.lower():
            return False
        meses = ['enero','febrero','marzo','abril','mayo','junio','julio','agosto','septiembre','octubre','noviembre','diciembre']
        if any(mes in txt.lower() for mes in meses):
            return False
        if txt.count(' ') > 3:
            return False
        return True

    try:
        res = requests.get(url_nota, timeout=10)
        soup = BeautifulSoup(res.text, 'html.parser')

        # 1. Cuerpo completo
        cuerpo = soup.find('div', class_='jl_content')
        texto_completo = cuerpo.get_text(separator="\n", strip=True) if cuerpo else ''

        # 2. Vistas
        vistas = soup.find(class_='jl_view_options')
        vistas_texto = vistas.get_text(strip=True) if vistas else ''

        # 3. Categorías útiles desde jl_shead_tpl_txt
        categorias = []
        shead = soup.find(class_='jl_shead_tpl_txt')
        if shead:
            for el in shead.find_all(['span', 'a']):
                txt = el.get_text(strip=True)
                if es_categoria_util(txt, titulo_nota):
                    categorias.append(txt)
        # Sacar duplicados
        categorias_unicas = ', '.join(sorted(set(categorias)))

        return {
            'descripcion': texto_completo,
            'vistas': vistas_texto,
            'categorias': categorias_unicas
        }
    except Exception as e:
        print(f"Error en {url_nota}: {e}")
        return {
            'descripcion': '',
            'vistas': '',
            'categorias': ''
        }


In [ ]:
import pandas as pd
import time
import os

# --- Parámetros generales ---
secciones = {
    'politica': 'https://www.eldiario.com.ar/category/actualidad/politica/',
    'economia': 'https://www.eldiario.com.ar/category/actualidad/economia/'
}
N_PAGINAS = 500
tmp_listado_file = "tmp_listado.csv"
tmp_detalle_file = "tmp_detalles.csv"

# Funciones que ya deberías tener definidas:
# - scrapear_listado_eldiario(url_base, seccion, n_pag)
# - scrapear_detalle_eldiario(url)

# --- 1. Scrapear listado (con recuperación) ---
resultados = []

# Si ya hay un listado parcial, cargarlo
if os.path.exists(tmp_listado_file):
    df_existente = pd.read_csv(tmp_listado_file)
    resultados = df_existente.to_dict('records')
    print(f"✅ Recuperado listado parcial con {len(resultados)} filas.")
else:
    df_existente = pd.DataFrame(columns=["enlace", "pagina", "seccion"])

for seccion, url_base in secciones.items():
    # Detectar última página scrappeada
    paginas_existentes = df_existente.query(f"seccion == '{seccion}'")["pagina"].dropna().unique()
    ultima_pagina = int(paginas_existentes.max()) if len(paginas_existentes) > 0 else 0

    for n_pag in range(ultima_pagina + 1, N_PAGINAS + 1):
        try:
            datos = scrapear_listado_eldiario(url_base, seccion, n_pag)
            for d in datos:
                d["pagina"] = n_pag
                d["seccion"] = seccion
            resultados.extend(datos)
        except Exception as e:
            print(f"⚠️ Error en {seccion}, página {n_pag}: {e}")
            continue
        if n_pag % 10 == 0:
            pd.DataFrame(resultados).drop_duplicates(subset=['enlace']).to_csv(tmp_listado_file, index=False)
            print(f"💾 Checkpoint listado guardado ({len(resultados)} filas)")
        time.sleep(1)

# Guardar listado final
df = pd.DataFrame(resultados).drop_duplicates(subset=['enlace']).reset_index(drop=True)
df.to_csv(tmp_listado_file, index=False)
print("📦 Listado final guardado")

# --- 2. Scrapear detalle de cada nota (con recuperación) ---
detalles = []

# Si ya hay un parcial, cargarlo
if os.path.exists(tmp_detalle_file):
    detalles = pd.read_csv(tmp_detalle_file).to_dict('records')
    print(f"✅ Recuperado detalle parcial con {len(detalles)} filas.")

# Determinar desde dónde retomar
enlaces_scrapeados = {d['enlace'] for d in detalles if 'enlace' in d}
pendientes = df[~df['enlace'].isin(enlaces_scrapeados)].copy()

print(f"🕵️‍♀️ Noticias pendientes: {len(pendientes)}")

for idx, row in pendientes.iterrows():
    url_nota = row['enlace']
    try:
        detalle = scrapear_detalle_eldiario(url_nota)
        detalle["enlace"] = url_nota  # clave para merge
        detalles.append(detalle)
    except Exception as e:
        print(f"⚠️ Error en detalle idx {idx}, url {url_nota}: {e}")
        detalles.append({'enlace': url_nota, 'error': str(e)})
    if (len(detalles) % 20) == 0:
        pd.DataFrame(detalles).to_csv(tmp_detalle_file, index=False)
        print(f"💾 Checkpoint detalles guardado ({len(detalles)} filas)")
    print(f"[{idx+1}/{len(df)}] {url_nota}")
    time.sleep(0.5)

# Guardar detalle final
df_detalle = pd.DataFrame(detalles)
df_detalle.to_csv(tmp_detalle_file, index=False)
print("📦 Detalle final guardado")

# --- 3. Unir y guardar final ---
df_final = pd.merge(df, df_detalle, on="enlace", how="left")
df_final_eldiario = df_final.drop_duplicates(subset=['enlace']).reset_index(drop=True)
df_final_eldiario.to_csv('../data/raw/eldiario_politica_economia_completo.csv', index=False)
print("✅ ¡Scraping completo y guardado!")

# Opcional: borrar temporales
# os.remove(tmp_listado_file)
# os.remove(tmp_detalle_file)


✅ Recuperado listado parcial con 2570 filas.
Scrapeando politica página 176 -> https://www.eldiario.com.ar/category/actualidad/politica/page/176/
Scrapeando politica página 177 -> https://www.eldiario.com.ar/category/actualidad/politica/page/177/
Scrapeando politica página 178 -> https://www.eldiario.com.ar/category/actualidad/politica/page/178/
Scrapeando politica página 179 -> https://www.eldiario.com.ar/category/actualidad/politica/page/179/
Scrapeando politica página 180 -> https://www.eldiario.com.ar/category/actualidad/politica/page/180/
💾 Checkpoint listado guardado (2600 filas)
Scrapeando politica página 181 -> https://www.eldiario.com.ar/category/actualidad/politica/page/181/
Scrapeando politica página 182 -> https://www.eldiario.com.ar/category/actualidad/politica/page/182/
Scrapeando politica página 183 -> https://www.eldiario.com.ar/category/actualidad/politica/page/183/
Scrapeando politica página 184 -> https://www.eldiario.com.ar/category/actualidad/politica/page/184/
Scr

## Análisis digital

In [ ]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

# --- Parámetros generales ---
secciones = {
    'provinciales': 'https://www.analisisdigital.com.ar/provinciales',
    'economia': 'https://www.analisisdigital.com.ar/economia'
}
N_PAGINAS = 200
tmp_file = "tmp_analisis.csv"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}

# --- Recuperar scraping parcial si existe ---
resultados = []
enlaces_existentes = set()

if os.path.exists(tmp_file):
    print("🔁 Recuperando scraping parcial...")
    df_parcial = pd.read_csv(tmp_file)
    resultados = df_parcial.to_dict('records')
    enlaces_existentes = set(df_parcial['enlace'])
    print(f"✅ Recuperadas {len(resultados)} noticias.")
else:
    df_parcial = pd.DataFrame(columns=['seccion', 'pagina', 'enlace'])

# --- Scraping principal ---
for seccion, url_base in secciones.items():
    paginas_scrapeadas = df_parcial[df_parcial['seccion'] == seccion]['pagina'].dropna().unique()
    ultima_pagina = int(paginas_scrapeadas.max()) if len(paginas_scrapeadas) > 0 else 0

    for n_pag in range(ultima_pagina + 1, N_PAGINAS + 1):
        url = url_base if n_pag == 1 else f"{url_base}?page={n_pag-1}"
        print(f"\n🗞️ Scrapeando {seccion}, página {n_pag}: {url}")
        t0 = time.time()

        try:
            res = requests.get(url, headers=headers, timeout=10)
            soup = BeautifulSoup(res.text, "html.parser")
            main_content = soup.find('div', class_='body')
            items = main_content.find_all('div', class_='views-row') if main_content else []

            print(f"   🔎 Noticias encontradas: {len(items)}")
            cuenta = 0

            for i, item in enumerate(items):
                try:
                    a_tag = item.find('a', href=True)
                    if not a_tag:
                        continue
                    enlace = a_tag['href']
                    if f"/{seccion}/" not in enlace:
                        continue
                    if not enlace.startswith('http'):
                        enlace = "https://www.analisisdigital.com.ar" + enlace
                    if enlace in enlaces_existentes:
                        continue

                    h2_tag = item.find('h2')
                    h3_tag = item.find('h3')
                    titulo = h2_tag.get_text(strip=True) if h2_tag else (h3_tag.get_text(strip=True) if h3_tag else '')

                    # Entrar al detalle
                    try:
                        res_nota = requests.get(enlace, headers=headers, timeout=10)
                        soup_nota = BeautifulSoup(res_nota.text, "html.parser")
                        fecha_tag = soup_nota.find('div', class_=lambda x: x and 'field--name-node-post-date' in x)
                        fecha = fecha_tag.get_text(strip=True) if fecha_tag else ''
                        cuerpo_div = soup_nota.find('div', class_=lambda x: x and 'body-noticia' in x)
                        parrafos = [p.get_text(strip=True) for p in cuerpo_div.find_all('p')] if cuerpo_div else []
                        contenido = "\n".join(parrafos)
                    except Exception as e:
                        print(f"      ⚠️ ERROR al entrar a nota: {e}")
                        fecha = ''
                        contenido = ''

                    resultados.append({
                        'seccion': seccion,
                        'pagina': n_pag,
                        'enlace': enlace,
                        'titulo': titulo,
                        'fecha': fecha,
                        'contenido': contenido
                    })
                    enlaces_existentes.add(enlace)
                    cuenta += 1
                    print(f"   ✅ [{cuenta}] {titulo[:60]}...")

                    if len(resultados) % 25 == 0:
                        pd.DataFrame(resultados).to_csv(tmp_file, index=False)
                        print(f"💾 Checkpoint guardado: {len(resultados)} noticias")

                    time.sleep(0.5)

                except Exception as e:
                    print(f"   ⚠️ ERROR procesando item {i} en {seccion}, página {n_pag}: {e}")

            print(f"  ✅ Noticias válidas en página {n_pag}: {cuenta}")

        except Exception as e:
            print(f"⛔ ERROR en página {url}: {e}")
        
        t1 = time.time()
        print(f"⏱️ Tiempo total: {t1 - t0:.1f} segundos")
        time.sleep(1)

# --- Guardado final ---
df = pd.DataFrame(resultados)
df_final_analisis = df.drop_duplicates(subset=['enlace']).reset_index(drop=True)
df_final_analisis.to_csv('../data/raw/analisis_provinciales_economia_completo.csv', index=False)
print(f"\n✅ Total de noticias finales: {len(df_final_analisis)}")
print(df_final_analisis[['seccion', 'pagina', 'enlace', 'titulo']].head(5))

# --- Borrá temporal si querés ---
# os.remove(tmp_file)



Scrapeando provinciales página 1: https://www.analisisdigital.com.ar/provinciales
  Noticias encontradas en página 1: 10 (antes del filtro)
   [1] Título: La vuelta a clases, en medio de un clima de tensió... | Enlace: https://www.analisisdigital.com.ar/provinciales/2025/07/20/la-vuelta-clases-en-medio-de-un-clima-de-tension
   [2] Título: Mauro Díaz Chaves, intendente de Aldea San Antonio... | Enlace: https://www.analisisdigital.com.ar/provinciales/2025/07/20/mauro-diaz-chaves-intendente-de-aldea-san-antonio-donde-esta-el-estado-que
   [3] Título: Hondo pesar por el fallecimiento del reconocido ab... | Enlace: https://www.analisisdigital.com.ar/provinciales/2025/07/19/hondo-pesar-por-el-fallecimiento-del-reconocido-abogado-jorge-campos
   [4] Título: Emisión de nueva deuda provincial: cuánto dinero s... | Enlace: https://www.analisisdigital.com.ar/provinciales/2025/07/19/emision-de-nueva-deuda-provincial-cuanto-dinero-salio-buscar-entre-rios-y
   [5] Título: La Cámara de Farmacias ex

## El Heraldo

In [68]:
import time
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

# -- CONFIG --
SECCIONES = {
    'politica': 'https://www.elheraldo.com.ar/noticias/politica',
    'economia': 'https://www.elheraldo.com.ar/noticias/economia'
}

N_PAGINAS = 200

# -- SETUP SELENIUM --
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

resultados = []
tmp_file = "tmp_heraldo.csv"

# -- Recuperar parciales si existen --
if os.path.exists(tmp_file):
    print("Recuperando scraping parcial...")
    resultados = pd.read_csv(tmp_file).to_dict('records')
    print(f"Recuperadas {len(resultados)} noticias.")

for seccion, url_base in SECCIONES.items():
    for n_pag in range(1, N_PAGINAS+1):
        url = url_base if n_pag == 1 else f"{url_base}?page={n_pag}"
        print(f"\nScrapeando {seccion} página {n_pag}: {url}")
        try:
            driver.get(url)
            time.sleep(2)  # Espera que cargue JS

            grid = driver.find_element(By.CLASS_NAME, 'NotesGroup_gridContainer__0ouzS')
            links = grid.find_elements(By.TAG_NAME, 'a')

            enlaces_vistos = set()
            noticias = []
            for a_tag in links:
                enlace = a_tag.get_attribute('href')
                if not enlace or enlace in enlaces_vistos:
                    continue
                enlaces_vistos.add(enlace)
                titulo = a_tag.get_attribute('title') or a_tag.text.strip()
                if not titulo or not enlace.startswith('http'):
                    continue
                noticias.append({'seccion': seccion, 'pagina': n_pag, 'enlace': enlace, 'titulo': titulo})

            print(f"  Noticias únicas encontradas en esta página: {len(noticias)}")

            # Extraemos contenido de cada nota
            for nota in noticias:
                enlace = nota['enlace']
                try:
                    driver.get(enlace)
                    time.sleep(1.5)
                    # Fecha
                    fecha = ''
                    try:
                        fecha_elem = driver.find_element(By.CLASS_NAME, 'NoteHeader_date__4aby9')
                        fecha = fecha_elem.text.strip()
                    except:
                        pass

                    # Copete
                    copete = ''
                    try:
                        copete_elem = driver.find_element(By.CLASS_NAME, 'NoteCoverImage_copete__VwAAE')
                        copete = copete_elem.text.strip()
                    except:
                        pass

                    # Contenido
                    contenido = ''
                    try:
                        cuerpo = driver.find_element(By.CLASS_NAME, 'NoteBody_wrapper__clqh4')
                        parrafos = cuerpo.find_elements(By.TAG_NAME, 'p')
                        contenido = '\n'.join([p.text.strip() for p in parrafos if p.text.strip()])
                    except:
                        pass

                except Exception as e:
                    print(f"    [ERROR] Falló al entrar a la nota {enlace}: {e}")
                    fecha, copete, contenido = '', '', ''

                resultados.append({
                    'seccion': nota['seccion'],
                    'pagina': nota['pagina'],
                    'enlace': enlace,
                    'titulo': nota['titulo'],
                    'fecha': fecha,
                    'copete': copete,
                    'contenido': contenido
                })
                # Checkpoint cada 20 noticias
                if len(resultados) % 20 == 0:
                    pd.DataFrame(resultados).to_csv(tmp_file, index=False)
                    print(f"   [Checkpoint: {len(resultados)} noticias guardadas]")
                time.sleep(0.3)
        except Exception as e:
            print(f"  [ERROR] No se encontró el grid de noticias en {url}: {e}")
        # Checkpoint por página
        pd.DataFrame(resultados).to_csv(tmp_file, index=False)
        print(f"   [Checkpoint (por página): {len(resultados)} noticias guardadas]")

# -- Guardar resultados --
driver.quit()
df = pd.DataFrame(resultados)
df = df.drop_duplicates(subset=['enlace']).reset_index(drop=True)
print(df[['seccion', 'pagina', 'titulo', 'fecha', 'enlace']])
df.to_csv('../data/raw/noticias_heraldo_politica_economia.csv', index=False)

# -- Opcional: borrar temporal
# os.remove(tmp_file)
print("Scraping de El Heraldo completado y guardado.")



Scrapeando politica página 1: https://www.elheraldo.com.ar/noticias/politica
  Noticias únicas encontradas en esta página: 18
   [Checkpoint (por página): 18 noticias guardadas]

Scrapeando politica página 2: https://www.elheraldo.com.ar/noticias/politica?page=2
  Noticias únicas encontradas en esta página: 18
   [Checkpoint: 20 noticias guardadas]
   [Checkpoint (por página): 36 noticias guardadas]

Scrapeando politica página 3: https://www.elheraldo.com.ar/noticias/politica?page=3
  Noticias únicas encontradas en esta página: 18
   [Checkpoint: 40 noticias guardadas]
   [Checkpoint (por página): 54 noticias guardadas]

Scrapeando politica página 4: https://www.elheraldo.com.ar/noticias/politica?page=4
  Noticias únicas encontradas en esta página: 18
   [Checkpoint: 60 noticias guardadas]
   [Checkpoint (por página): 72 noticias guardadas]

Scrapeando politica página 5: https://www.elheraldo.com.ar/noticias/politica?page=5
  Noticias únicas encontradas en esta página: 18
   [Checkpoi